論文 
https://arxiv.org/abs/2003.03808v3  
GitHub  
https://github.com/adamian98/pulse  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/PULSE_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# GitHubからPULSEのソースコードを取得

In [ ]:
%cd /content/
!git clone https://github.com/adamian98/pulse.git

# ライブラリのインストール

In [ ]:
!apt install ffmpeg

# ライブラリのインポート

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import glob
import os
from moviepy.editor import *
from moviepy.video.fx.resize import resize

# テスト画像のセットアップ

In [ ]:
%cd /content/pulse

!rm -rf ./org_img
!mkdir ./org_img
%cd /content/pulse/org_img

image_type ='sample' #@param ['sample', 'upload']
if image_type == 'sample':
  !wget https://www.pakutaso.com/shared/img/thumb/YK0I9A6188_TP_V4.jpg
  file_name = "YK0I9A6188_TP_V4.jpg"
else:
  uploaded = files.upload()
  uploaded = list(uploaded.keys())
  file_name = uploaded[0]


# 低解像度顔画像生成
16x16, 64x64, 128x128の3種を生成

In [ ]:
%cd /content/pulse/
# 顔部分を切り出した低解像度画像出力先
img_align_16 = "./img_align_16"
img_align_64 = "./img_align_64"
img_align_128 = "./img_align_128"

!rm -rf {img_align_16}
!rm -rf {img_align_64}
!rm -rf {img_align_128}

# 16x16
!python align_face.py \
  -input_dir ./org_img \
  -output_dir {img_align_16} \
  -output_size 16 \
  -seed 12 \
# 64x64
!python align_face.py \
  -input_dir ./org_img \
  -output_dir {img_align_64} \
  -output_size 64 \
  -seed 12 \
# 128x128
!python align_face.py \
  -input_dir ./org_img \
  -output_dir {img_align_128} \
  -output_size 128 \
  -seed 12 \

## 低解像度顔画像の表示

In [ ]:
align_16 = glob.glob(img_align_16 + "/*.png") 
image_16 = Image.open(align_16[0]).convert("RGB")
align_64 = glob.glob(img_align_64 + "/*.png") 
image_64 = Image.open(align_64[0]).convert("RGB")
align_128 = glob.glob(img_align_128 + "/*.png") 
image_128 = Image.open(align_128[0]).convert("RGB")

fig = plt.figure(num=None, figsize=(12, 5))
# 16x16
ax = fig.add_subplot(1, 3, 1, xticks=[], yticks=[])
plt.imshow(image_16)
ax.set_title("16x16")
# 64x64
ax = fig.add_subplot(1, 3, 2, xticks=[], yticks=[])
plt.imshow(image_64)
ax.set_title("64x64")
# 128x128
ax = fig.add_subplot(1, 3, 3, xticks=[], yticks=[])
plt.imshow(image_128)
ax.set_title("128x128")


# 超解像
16x16, 64x64, 128x128の3種をそれぞれ1024x1024へ変換

In [ ]:
%cd /content/pulse/
img_sr_16 = "./img_sr_16"
img_sr_64 = "./img_sr_64"
img_sr_128 = "./img_sr_128"
steps = 500

!rm -rf {img_sr_16}
!rm -rf {img_sr_64}
!rm -rf {img_sr_128}

# 16x16
!python run.py \
  -input_dir {img_align_16} \
  -output_dir {img_sr_16} \
  -duplicates 1 \
  -seed 12 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate
# 64x64
!python run.py \
  -input_dir {img_align_64} \
  -output_dir {img_sr_64} \
  -duplicates 1 \
  -seed 12 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate
# 128x128
!python run.py \
  -input_dir {img_align_128} \
  -output_dir {img_sr_128} \
  -duplicates 1 \
  -seed 12 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate

## 超解像結果の表示

In [ ]:
file_name_wo_ext = os.path.splitext(os.path.basename(file_name))[0]
sr_16_file_name = img_sr_16 + "/" + file_name_wo_ext + "_0/HR/" + file_name_wo_ext + "_0_" + str(steps-1) + ".png"
sr_64_file_name = img_sr_64 + "/" + file_name_wo_ext + "_0/HR/" + file_name_wo_ext + "_0_" + str(steps-1) + ".png"
sr_128_file_name = img_sr_128 + "/" + file_name_wo_ext + "_0/HR/" + file_name_wo_ext + "_0_" + str(steps-1) + ".png"
sr_image_16 = Image.open(sr_16_file_name).convert("RGB")
sr_image_64 = Image.open(sr_64_file_name).convert("RGB")
sr_image_128 = Image.open(sr_128_file_name).convert("RGB")


fig = plt.figure(num=None, figsize=(12, 5))
# 16x16
ax = fig.add_subplot(2, 3, 1, xticks=[], yticks=[])
plt.imshow(image_16)
ax.set_title("16x16")
ax = fig.add_subplot(2, 3, 4, xticks=[], yticks=[])
plt.imshow(sr_image_16)
ax.set_title("sr 16x16")
# 64x64
ax = fig.add_subplot(2, 3, 2, xticks=[], yticks=[])
plt.imshow(image_64)
ax.set_title("64x64")
ax = fig.add_subplot(2, 3, 5, xticks=[], yticks=[])
plt.imshow(sr_image_64)
ax.set_title("sr 64x64")
ax = fig.add_subplot(2, 3, 3, xticks=[], yticks=[])
plt.imshow(image_128)
ax.set_title("128x128")
ax = fig.add_subplot(2, 3, 6, xticks=[], yticks=[])
plt.imshow(sr_image_128)
ax.set_title("sr 128x128")

## 探索の様子を動画に出力

In [ ]:
sr_128_frames = img_sr_128 + "/" + file_name_wo_ext + "_0/HR/" + file_name_wo_ext + "_0_%02d.png"
transfer_video = img_sr_128 + "/" + file_name_wo_ext + "_0/sr_128.mp4"

!ffmpeg -i {sr_128_frames} -c:v libx264 -vf "fps=25,format=yuv420p" {transfer_video}

transfer_clip = VideoFileClip(transfer_video)
# 表示用にリサイズ
transfer_clip_resize = resize(transfer_clip, height=400)
transfer_clip_resize.ipython_display()

# テスト動画のセットアップ
本レシピで使用している動画は下記です。  
https://pixabay.com/ja/videos/%E6%84%9F%E6%83%85-%E7%94%B7%E6%80%A7%E3%82%92%E8%A6%8B-%E4%BA%BA%E9%96%93-%E4%BA%BA-44440/

In [ ]:
%cd /content/pulse
!rm -rf ./origin_video
!mkdir -p ./origin_video/frames
%cd /content/pulse/origin_video

uploaded = files.upload()
uploaded = list(uploaded.keys())
video_name = uploaded[0]

clip = VideoFileClip(video_name)
# 2~3秒までを切り出し
sub_clip = clip.subclip(2, 3)
sub_clip.write_videofile("src-video.mp4")
video_name = "src-video.mp4"

# 表示用にリサイズ
clip_resize = resize(sub_clip, height=240)
clip_resize.ipython_display()

# テスト動画をフレーム分割

In [ ]:
!ffmpeg -i {video_name} /content/pulse/origin_video/frames/src-video-frame-%d.png

# 低解像度顔画像生成

In [ ]:
%cd /content/pulse/
# 顔部分を切り出した低解像度画像出力先
video_align_dir = "./video_input_face"

!rm -rf {video_align_dir}

!python align_face.py \
  -input_dir ./origin_video/frames \
  -output_dir {video_align_dir} \
  -output_size 64 \
  -seed 1234 \

video_align_imgs = glob.glob(video_align_dir + "/*.png") 
image = Image.open(video_align_imgs[0]).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 超解像(動画)
64x64から1024x1024へ変換

In [ ]:
%cd /content/pulse/
sr_video_result = "./sr_video_result"
steps = 20

!rm -rf {sr_video_result}

!python run.py \
  -input_dir {video_align_dir} \
  -output_dir {sr_video_result} \
  -duplicates 1 \
  -seed 30 \
  -noise_type trainable \
  -steps {steps} \
  -save_intermediate

# frame画像を収集
各フレーム画像の超解像結果を一つのディレクトリ配下にコピー

In [ ]:
%cd /content/pulse/
sr_video_result_frames = sr_video_result + "/frames"
!mkdir -p {sr_video_result_frames}

results = glob.glob(sr_video_result + "/*/HR/*_0_" + str(steps-1) + ".png")
print(results)

for result in results:
  !cp {result} {sr_video_result_frames}


# frame画像を動画に変換

In [ ]:
src_frames = sr_video_result_frames + "/src-video-frame-%0d_0_" + str(steps-1) + ".png"
dst_video = sr_video_result + "/out.mp4"

!ffmpeg -i {src_frames} -c:v libx264 -vf "fps=25,format=yuv420p" {dst_video}

clip = VideoFileClip(dst_video)

# 表示用にリサイズ
clip_resize = resize(clip, height=400)
clip_resize.ipython_display()